In [28]:
import pandas as pd

In [29]:
df_master=pd.read_csv("../../data/silver/NCDB_Toront_holiday_merged/toronto_ncdb_master.csv")
weekly_weather=pd.read_csv("../../data/silver/weather/toronto_weekly_weather.csv")

In [30]:
df_master['ds'] = pd.to_datetime(df_master['ds'])

In [31]:
weekly_weather.head()

,time,precip_mm,snow_depth_cm,min_temp
0,2014-01-06,20.8,1.2,-23.8
1,2014-01-13,12.0,1.2,-24.2
2,2014-01-20,6.0,0.4,-17.3
3,2014-01-27,7.4,0.9,-23.5
4,2014-02-03,12.8,1.4,-20.3


In [32]:
df_master.head()

,Unnamed: 0,ds,y,national_index,week_of_year,is_holiday
0,0,2014-01-06,136,28539.0,2,0
1,1,2014-01-13,136,3090.0,3,0
2,2,2014-01-20,155,0.0,4,0
3,3,2014-01-27,174,0.0,5,0
4,4,2014-02-03,196,10353.0,6,0


In [33]:
print(f"Master 'ds' type: {df_master['ds'].dtype}")
print(f"Weather Index type: {weekly_weather.index.dtype}")

Master 'ds' type: datetime64[ns]
Weather Index type: int64


In [34]:
print(weekly_weather.head())
print(df_master.head())

         time  precip_mm  snow_depth_cm  min_temp
0  2014-01-06       20.8            1.2     -23.8
1  2014-01-13       12.0            1.2     -24.2
2  2014-01-20        6.0            0.4     -17.3
3  2014-01-27        7.4            0.9     -23.5
4  2014-02-03       12.8            1.4     -20.3
   Unnamed: 0         ds    y  national_index  week_of_year  is_holiday
0           0 2014-01-06  136         28539.0             2           0
1           1 2014-01-13  136          3090.0             3           0
2           2 2014-01-20  155             0.0             4           0
3           3 2014-01-27  174             0.0             5           0
4           4 2014-02-03  196         10353.0             6           0


In [37]:
print( 'time' in weekly_weather.columns)  # Check if 'time' is a column in weekly_weather

True


In [38]:
weekly_weather['ds'] = pd.to_datetime(weekly_weather['time'])

In [39]:
weekly_weather = weekly_weather.set_index('ds')

In [40]:
weekly_weather.head()

,index,time,precip_mm,snow_depth_cm,min_temp
ds,,,,,
2014-01-06,0,2014-01-06,20.8,1.2,-23.8
2014-01-13,1,2014-01-13,12.0,1.2,-24.2
2014-01-20,2,2014-01-20,6.0,0.4,-17.3
2014-01-27,3,2014-01-27,7.4,0.9,-23.5
2014-02-03,4,2014-02-03,12.8,1.4,-20.3


In [ ]:
weekly_weather = weekly_weather[['precip_mm', 'snow_depth_cm', 'min_temp']]




--- WEATHER DATA FIXED ---
            precip_mm  snow_depth_cm  min_temp
ds                                            
2014-01-06       20.8            1.2     -23.8
2014-01-13       12.0            1.2     -24.2
2014-01-20        6.0            0.4     -17.3
2014-01-27        7.4            0.9     -23.5
2014-02-03       12.8            1.4     -20.3


In [43]:
df_master = df_master.sort_values('ds')
weekly_weather = weekly_weather.sort_index()

In [44]:
df_final = pd.merge_asof(
    df_master, 
    weekly_weather, 
    left_on='ds', 
    right_index=True, 
    direction='nearest', 
    tolerance=pd.Timedelta('7 days')
)

In [46]:

print(df_final[['ds', 'y', 'precip_mm', 'snow_depth_cm']].head())

          ds    y  precip_mm  snow_depth_cm
0 2014-01-06  136       20.8            1.2
1 2014-01-13  136       12.0            1.2
2 2014-01-20  155        6.0            0.4
3 2014-01-27  174        7.4            0.9
4 2014-02-03  196       12.8            1.4


In [48]:
df_final.notnull().sum()

Unnamed: 0        614
ds                614
y                 614
national_index    614
week_of_year      614
is_holiday        614
precip_mm         614
snow_depth_cm     228
min_temp          614
dtype: int64

In [49]:
df_final['snow_depth_cm'] = df_final['snow_depth_cm'].fillna(0)

In [50]:
df_final.notnull().sum()

Unnamed: 0        614
ds                614
y                 614
national_index    614
week_of_year      614
is_holiday        614
precip_mm         614
snow_depth_cm     614
min_temp          614
dtype: int64

In [54]:
import os
os.makedirs("../../data/silver/master_weather_merged", exist_ok=True)
df_final.to_csv("../../data/silver/master_weather_merged/toronto_master_weather_merged.csv", index=False)   